In [ ]:
import numpy as np
import math
import pandas as pd
import glob
import os
from seabird.cnv import fCNV

In [2]:
locations = pd.read_csv(r'C:\Data\Ocean\locations.csv')
format1_list = []
for root, dirs, files in os.walk(r"C:\Data\Ocean\Problem files\Old _format"):
    for file in files:
        if file.endswith(".cnv"):
            print(os.path.join(root, file))
            path = os.path.join(root, file)
            date = file.split('_')[3]
            instrument = file.split('_')[4]
            f=open(path,"r")
            lines=f.readlines()
            data_list = []
            index = [0,1,2,3,4,5,6,7,8,9,10,11,12] #index the variables that you want to read into the data frame
            for i in index:
                list = []
                for x in lines:
                    list.append(x.split('\t')[i])
                df = pd.DataFrame(list[1:], columns = [list[0]])
                data_list.append(df)
                f.close()
            data = pd.concat(data_list, axis = 1)
            data['date'] = date
            data['instrument'] = instrument
            data.Depth = pd.to_numeric(data['Depth'])
            data.Conductivity = pd.to_numeric(data['Conductivity'])
            data.Temperature = pd.to_numeric(data['Temperature'])
            data.Salinity = pd.to_numeric(data['Salinity'])
            data.Oxygen = pd.to_numeric(data['Oxygen'])
            data.pH = pd.to_numeric(data['pH'])
            data.Chlorophyll = pd.to_numeric(data['Chlorophyll'])
            data.Turbidity = pd.to_numeric(data['Turbidity'])
            data = data[data.Depth >= 0]
            #group data into defined bins
            cut_labels = ['0-2', '2-10', '10-20', '20-30','30-40','40-50','50-60','60-70', '70-80']
            cut_bins = [0, 2, 10, 20, 30, 40, 50, 60, 70, 80]
            depth_class = pd.cut(data['Depth'], bins=cut_bins, labels=cut_labels).rename('depth_class')
            df=pd.concat([data,depth_class], axis=1).sort_values('Depth', ascending=True)
            format1_list.append(df)
format1_data = pd.concat(format1_list)
df = format1_data.drop(columns = ['Station', 'DecDegS', 'DecDegE'], axis = 1)
df = df.rename(columns = {'instrument':'Station'})
format1_df=pd.merge(df,locations[['Station','DecDegS','DecDegE']], on = 'Station')
print(format1_df)

C:\Data\Ocean\Problem files\Old _format\SAEON_ABSS_CTP_20180220_PELTERP1_FV00_CTDPRO.cnv
C:\Data\Ocean\Problem files\Old _format\SAEON_ABSS_CTP_20180220_PELTERP4_FV00_CTDPRO.cnv
C:\Data\Ocean\Problem files\Old _format\SAEON_ABSS_CTP_20181025_PELTERP7_FV00_CTDPRO.cnv
      Conductivity  Temperature   Depth  Salinity   Oxygen     pH  \
0          48360.0      20.2958   0.029   35.1147  11.6217  9.022   
1          54252.0      20.4554   0.150   39.8187   5.9309  8.700   
2          48353.0      20.2943   0.163   35.1101  11.6294  9.020   
3          48353.0      20.2929   0.264   35.1119  11.6281  9.022   
4          48427.0      20.4276   0.273   35.0597   6.1180  8.800   
...            ...          ...     ...       ...      ...    ...   
3273       40200.0      12.1854  60.382   35.0322   4.2388  8.585   
3274       40200.0      12.1849  60.421   35.0341   4.4797  8.581   
3275       40200.0      12.1853  60.435   35.0319   4.2185  8.582   
3276       40200.0      12.1848  60.457   3

In [3]:
format2_list = []
for root, dirs, files in os.walk(r"C:\Data\Ocean\Algoa Bay Complete Dataset\PELTER_CTD"):
    for file in files:
        if file.endswith(".cnv"):
            path = os.path.join(root, file)
            print(path)
            #get the date and station from the file path
            date = file.split('_')[3]
            Station = file.split('_')[4]
            #open the file using pySeabird
            profile = fCNV(path)
            #get the variables from the CNV header
            vars = profile.keys()
            #create a list of the names you would like to rename the column headings to.
            variables = ['Conductivity', 'Temperature', 'Depth', 'Salinity', 'Oxygen', 'pH', 'Chlorophyll', 'Turbidity', 'Bottles', 'Julian', 'Scan', 'Pressure', 'flag', 'time']
            #build a dataframe from the .cnv files
            list = []
            for i in vars:
                df = pd.DataFrame(data = profile[i])
                list.append(df)
            data = pd.concat(list, axis = 1)
            data.columns = variables           
            data['date'] = date
            data['Station'] = Station
            #remove observations where the depth is negative
            data = data[data.Depth >= 0]
            #group data into defined bins
            cut_labels = ['0-2', '2-10', '10-20', '20-30','30-40','40-50','50-60','60-70', '70-80']
            cut_bins = [0, 2, 10, 20, 30, 40, 50, 60, 70, 80]
            depth_class = pd.cut(data['Depth'], bins=cut_bins, labels=cut_labels).rename('depth_class')
            df=pd.concat([data,depth_class], axis=1).sort_values('Depth', ascending=True)
            df=pd.merge(df,locations[['Station','DecDegS','DecDegE']], on = 'Station')
            format2_list.append(df)
format2_df= pd.concat(format2_list)
print(format2_df)

It was supposed to has 13 variables.


C:\Data\Ocean\OceanAccounts AlgoaBay Data\2018_PELTER_CTD\Run 1\2018-02_PELTER_CTD\RawData\SAEON_ABSS_CTP_20180220_PELTERP2_FV00_CTDPRO.cnv
C:\Data\Ocean\OceanAccounts AlgoaBay Data\2018_PELTER_CTD\Run 1\2018-02_PELTER_CTD\RawData\SAEON_ABSS_CTP_20180220_PELTERP3_FV00_CTDPRO.cnv


It was supposed to has 13 variables.
It was supposed to has 13 variables.


C:\Data\Ocean\OceanAccounts AlgoaBay Data\2018_PELTER_CTD\Run 1\2018-02_PELTER_CTD\RawData\SAEON_ABSS_CTP_20180220_PELTERP5_FV00_CTDPRO.cnv
C:\Data\Ocean\OceanAccounts AlgoaBay Data\2018_PELTER_CTD\Run 1\2018-02_PELTER_CTD\RawData\SAEON_ABSS_CTP_20180220_PELTERP6_FV00_CTDPRO.cnv


It was supposed to has 13 variables.


...

C:\Data\Ocean\OceanAccounts AlgoaBay Data\2019_PELTER_CTD\2019-12_PELTER_CTD\SAEON_ABSS_CTP_20191208_PELTERP5_FV00_CTDPRO.cnv
C:\Data\Ocean\OceanAccounts AlgoaBay Data\2019_PELTER_CTD\2019-12_PELTER_CTD\SAEON_ABSS_CTP_20191208_PELTERP6_FV00_CTDPRO.cnv


It was supposed to has 13 variables.


C:\Data\Ocean\OceanAccounts AlgoaBay Data\2019_PELTER_CTD\2019-12_PELTER_CTD\SAEON_ABSS_CTP_20191208_PELTERP7_FV00_CTDPRO.cnv


It was supposed to has 13 variables.
It was supposed to has 13 variables.


C:\Data\Ocean\OceanAccounts AlgoaBay Data\2019_PELTER_CTD\2019-12_PELTER_CTD\SAEON_ABSS_CTP_20191208_PELTERP8_FV00_CTDPRO.cnv
     Conductivity  Temperature   Depth  Salinity   Oxygen     pH  Chlorophyll  \
0         46918.0      18.8566   0.073   35.1322  12.2523  8.994       7.0314   
1         47178.0      18.9858   0.084   35.2407   8.4489  9.026       6.8750   
2         28538.0      19.0865   0.084   20.1253   7.2561  8.953       7.1031   
3         23441.0      19.1848   0.104   16.1807   7.2337  8.917       7.0558   
4         25197.0      19.0318   0.104   17.5768   7.5749  8.975       6.8116   
..            ...          ...     ...       ...      ...    ...          ...   
727       47764.0      19.4689  25.927   35.3091   4.7883  8.838       5.0469   
728       47746.0      19.4768  25.983   35.2877   4.9364  8.840       5.1705   
729       47753.0      19.4664  26.009   35.3022   4.8247  8.838       4.8867   
730       47749.0      19.4664  26.056   35.2992   4.9230  8.836

In [4]:
format3_list = []
for root, dirs, files in os.walk(r"C:\Data\Ocean\Problem files\big files"):
    for file in files:
        if file.endswith(".csv"):
            path = os.path.join(root, file)
            print(path)
            #get the date and station from the file path
            date = file.split('_')[3]
            Station = file.split('_')[4]
            df = pd.read_csv(path)
            data['date'] = date
            data['Station'] = Station
            #remove observations where the depth is negative
            data = data[data.Depth >= 0]
            #group data into defined bins
            cut_labels = ['0-2', '2-10', '10-20', '20-30','30-40','40-50','50-60','60-70', '70-80']
            cut_bins = [0, 2, 10, 20, 30, 40, 50, 60, 70, 80]
            depth_class = pd.cut(data['Depth'], bins=cut_bins, labels=cut_labels).rename('depth_class')
            df=pd.concat([data,depth_class], axis=1).sort_values('Depth', ascending=True)
            df=pd.merge(df,locations[['Station','DecDegS','DecDegE']], on = 'Station')
            format3_list.append(df)
format3_df= pd.concat(format3_list)
print(format3_df)

C:\Data\Ocean\Problem files\big files\SAEON_ABSS_CTP_20180911_PELTERP1_FV00_CTDPRO_processed.csv
     Conductivity  Temperature   Depth  Salinity  Oxygen     pH  Chlorophyll  \
0         48903.0      21.1449   0.057   34.8541  4.7226  8.497       1.5831   
1         48952.0      20.6460   0.058   35.3043  5.2387  8.861       2.9381   
2         48951.0      20.6396   0.100   35.3095  5.4464  8.858       2.8946   
3         48952.0      20.6306   0.114   35.3172  5.5011  8.857       2.9435   
4         48957.0      20.9983   0.127   35.0173  4.5996  8.617       1.6396   
..            ...          ...     ...       ...     ...    ...          ...   
727       47764.0      19.4689  25.927   35.3091  4.7883  8.838       5.0469   
728       47746.0      19.4768  25.983   35.2877  4.9364  8.840       5.1705   
729       47753.0      19.4664  26.009   35.3022  4.8247  8.838       4.8867   
730       47749.0      19.4664  26.056   35.2992  4.9230  8.836       4.7997   
731       47746.0      

In [5]:
df_list = [format1_df, format2_df, format3_df]
df = pd.concat(df_list)

In [6]:
#add some additional packages
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

#list the variables we want to display
sensor_list = ['Conductivity', 'Temperature', 'Salinity', 'Oxygen', 'pH', 'Chlorophyll', 'Turbidity', 'Pressure']

#Get a list of the Stations in alphabetical order
df = df.sort_values(by=['Station'])
stations = df['Station'].unique()

#Get a list of the dates we want to show
df['date'] = df['date'].replace('201900809', '20190809')
df = df.sort_values(by = 'date')
df['date'] = pd.to_datetime(df.date, format='%Y%m%d')
dates = df['date'].unique()
date_array = [str(i) for i in dates]
dates = [i[:10] for i in date_array]
print(dates)
date_mark = {i : dates[i] for i in range(0, 22)}

app = JupyterDash(__name__)
app.layout = html.Div([
#Heading
    html.H1("Hobo Data Visualisation App"),

#Subheading             
    html.H2("Dynamic Visualisation of variables"),

# Dropdown for the Station to chart
    html.Label([                                        
        "Station",                                     
        dcc.Dropdown(                                   
            id='stat-dropdown', clearable=False,         
            value='PELTERP1', options=[              
                {'label': st, 'value': st}
                for st in stations                    
            ])
    ]),

#timeslider to analyse time series
    html.Label([                                        
        "Date",
        dcc.RangeSlider(
            id = 'slider',
            marks = date_mark,
            min = 0,
            max = 22,
            value = [0,2])
    ]),

# Chart
    dcc.Graph(id='graph'),

# Dropdown for the variables to chart                              
    html.Label([                                        
        "Variable",                                     
        dcc.Dropdown(                                   
            id='var-dropdown', clearable=False,         
            value='Temperature', options=[              
                {'label': s, 'value': s}
                for s in sensor_list                    
            ])
    ]),

])

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),                          
     [Input("var-dropdown", "value"), 
     Input("stat-dropdown", "value"),
     Input("slider", "value") 
      ])

#define the function to update the graph based on the user selection
def update_figure(input1, input2, input3):
  #Filter the Data by station
  data = df[(df.date > dates[input3[0]]) & (df.date < dates[input3[1]])]
  #data = df.loc[df['Station'] ==input2]
  #update the plot
  fig = px.box(
        data.loc[data['Station'] ==input2], 
         x="depth_class", 
         y=input1,
         color = "depth_class",
         #color_continuous_scale= "Plasma",
         title= input2 
                                                                             
  )
  return fig
    
# Run app and display result inline in the notebook
app.run_server(mode='inline')

['2018-02-20', '2018-03-26', '2018-04-17', '2018-05-24', '2018-06-21', '2018-07-05', '2018-08-29', '2018-09-11', '2018-10-25', '2018-11-15', '2018-12-09', '2019-02-05', '2019-02-25', '2019-03-19', '2019-05-10', '2019-06-14', '2019-07-15', '2019-08-09', '2019-09-10', '2019-10-04', '2019-11-21', '2019-12-08']


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.